In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
prod=pd.read_csv(r"C:\Users\mohit\OneDrive\Desktop\Data_for_model\Instacart\products.csv")

prod

In [ ]:
order=pd.read_csv(r"C:\Users\mohit\OneDrive\Desktop\Data_for_model\Instacart\orders.csv")
order

In [ ]:
order_products=pd.read_csv(r"C:\Users\mohit\OneDrive\Desktop\Data_for_model\Instacart\order_products__prior.csv")
order_products

In [ ]:
df=order_products.merge(prod, on='product_id')
df

In [ ]:
df['order_id'].nunique()  # no of basket

In [ ]:
df['product_name'].nunique()

In [ ]:
df['product_name'].value_counts().head(20)

In [ ]:
basket_size=df.groupby('order_id')['product_id'].count()
basket_size

In [ ]:
basket_size.describe()

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
df['reordered'].mean()

In [ ]:
# Creating basket matrix

In [ ]:
df['product_name'].value_counts().head(5000)

In [ ]:
df1=df[df['order_id']<=100000]
df1

In [ ]:
df1['product_name'].value_counts()

In [ ]:
df1['product_name'].nunique()

In [ ]:
df1['product_name'].value_counts().head(1000)

In [ ]:
sample_orders=df['order_id'].drop_duplicates().sample(n=100000, random_state=42)


In [ ]:
df2=df[df['order_id'].isin(sample_orders)]
df2

In [ ]:
df2['order_id'].nunique()

In [ ]:
df2['product_name'].value_counts()

In [ ]:
bs=df2.groupby('order_id')['product_name'].count()
bs

In [ ]:
bs.describe()

In [ ]:
df2['product_name'].nunique()

In [ ]:
df2['product_name'].value_counts().head(3000)

In [ ]:
prod_count=df2['product_name'].value_counts()
prod_count

In [ ]:
freq_prod=prod_count[prod_count>=110].index
freq_prod

In [ ]:
df2=df2[df2['product_name'].isin(freq_prod)]

In [ ]:
df2

In [ ]:
df2['product_name'].value_counts()

In [ ]:
bs=df2.groupby('order_id')['product_name'].count()
bs

In [ ]:
bs.describe()

In [ ]:
df2

In [ ]:
basket=df2.groupby(['order_id','product_name'])['add_to_cart_order'].count().unstack().fillna(0)

basket

In [ ]:
df2.groupby('order_id')['product_name'].value_counts()

In [ ]:
bskt=basket.astype(bool)
bskt

In [ ]:
!pip install mlxtend

In [ ]:
import sys
sys.modules.pop('mlxtend', None)

In [ ]:
import sys
!{sys.executable} -m pip install mlxtend

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_items= apriori( bskt,min_support=0.01,use_colnames=True)

In [ ]:
frequent_items

In [ ]:
frequent_items.sort_values(by='support',ascending=False).head(20)

In [ ]:
frequent_items.sort_values(by='support',ascending=False).tail(20)

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(
    frequent_items,
    metric="lift",
    min_threshold=1
)

In [ ]:
rules

In [ ]:
rules = rules[
    (rules['confidence'] >= 0.2) &
    (rules['lift'] >= 1.2)
]

In [ ]:
rules.sort_values(by='lift', ascending=False)

In [ ]:
frequent_items.sort_values(by='support',ascending=False)

In [ ]:
frequent_items[
    frequent_items['itemsets'].apply(lambda x: len(x) == 2)
].sort_values(by='support', ascending=False)

In [ ]:
def recommend_products(product_name):

    recommendations = []

    for i in range(len(rules)):
        
        antecedent = list(rules.iloc[i]['antecedents'])
        consequent = list(rules.iloc[i]['consequents'])

        if product_name in antecedent:
            recommendations.extend(consequent)

    return list(set(recommendations))[:5]

In [ ]:
recommend_products('Organic Fuji Apple')

In [ ]:
rules[
    rules['antecedents'].apply(lambda x: 'Organic Fuji Apple' in x)
][['antecedents','consequents','confidence','lift']]

In [ ]:
len(rules)

In [ ]:
rules.iloc[0]['antecedents']

In [ ]:
rules

In [ ]:
for i in range(10):
    print(i)

In [ ]:
import pickle

pickle.dump(rules, open('association_rules.pkl','wb'))